In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer
from text_mining import TextMining

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/antoine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/antoine/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/antoine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data_train = pd.read_csv('../data/train_tweets.csv')
data_test = pd.read_csv('../data/test_tweets.csv')

In [3]:
data_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
tm = TextMining(data_train)
data_train = (
    tm.lowercase()
      .extract_target_char("#", "hashtags")
      .extract_target_char("@", "mentions")
      .extract_url()
      .clean_regex()
      .tokenize()
      .remove_stopwords()
      .apply_stemmer() #.apply_lemmatizer()
      .get_df()
)
data_train.head(15)

,id,keyword,location,text,target,hashtags,mentions,urls,tokens
0,1,NaN,NaN,our deeds are the reason of this may allah for...,1,earthquake,None,,"[deed, reason, may, allah, forgiv, us]"
1,4,NaN,NaN,forest fire near la ronge sask canada,1,None,None,,"[forest, fire, near, la, rong, sask, canada]"
2,5,NaN,NaN,all residents asked to shelter in place are ...,1,None,None,,"[resid, ask, shelter, place, notifi, offic, ev..."
3,6,NaN,NaN,13 000 people receive evacuation orders in cal...,1,wildfires,None,,"[13, 000, peopl, receiv, evacu, order, califor..."
4,7,NaN,NaN,just got sent this photo from ruby as smoke fr...,1,"alaska, wildfires",None,,"[got, sent, photo, rubi, smoke, pour, school]"
5,8,NaN,NaN,update california hwy 20 closed in both dir...,1,"rockyfire, cafire, wildfires",None,,"[updat, california, hwi, 20, close, direct, du..."
6,10,NaN,NaN,heavy rain causes flash flooding of streets in...,1,"flood, disaster",None,,"[heavi, rain, caus, flash, flood, street, mani..."
7,13,NaN,NaN,i m on top of the hill and i can see a fire in...,1,None,None,,"[top, hill, see, fire, wood]"
8,14,NaN,NaN,there s an emergency evacuation happening now ...,1,None,None,,"[emerg, evacu, happen, build, across, street]"
9,15,NaN,NaN,i m afraid that the tornado is coming to our a...,1,None,None,,"[afraid, tornado, come, area]"


In [5]:
mentions_count = data_train['mentions'].apply(lambda x: isinstance(x, str) and x.strip() != "").sum()
hashtags_count = data_train['hashtags'].apply(lambda x: isinstance(x, str) and x.strip() != "").sum()
urls_count = data_train['urls'].apply(lambda x: isinstance(x, str) and x.strip() != "").sum()

print(f"Mentions : {mentions_count}")
print(f"Hashtags : {hashtags_count}")
print(f"URLs : {urls_count}")

data_train[
    (data_train['mentions'].apply(lambda x: isinstance(x, str) and x.strip() != "")) |
    (data_train['hashtags'].apply(lambda x: isinstance(x, str) and x.strip() != "")) |
    (data_train['urls'].apply(lambda x: isinstance(x, str) and x.strip() != ""))
].head()


Mentions : 2009
Hashtags : 1743
URLs : 3971


,id,keyword,location,text,target,hashtags,mentions,urls,tokens
0,1,NaN,NaN,our deeds are the reason of this may allah for...,1,earthquake,None,,"[deed, reason, may, allah, forgiv, us]"
3,6,NaN,NaN,13 000 people receive evacuation orders in cal...,1,wildfires,None,,"[13, 000, peopl, receiv, evacu, order, califor..."
4,7,NaN,NaN,just got sent this photo from ruby as smoke fr...,1,"alaska, wildfires",None,,"[got, sent, photo, rubi, smoke, pour, school]"
5,8,NaN,NaN,update california hwy 20 closed in both dir...,1,"rockyfire, cafire, wildfires",None,,"[updat, california, hwi, 20, close, direct, du..."
6,10,NaN,NaN,heavy rain causes flash flooding of streets in...,1,"flood, disaster",None,,"[heavi, rain, caus, flash, flood, street, mani..."


In [6]:
data_train[data_train['mentions'].str.strip() != ""].head()

,id,keyword,location,text,target,hashtags,mentions,urls,tokens
0,1,NaN,NaN,our deeds are the reason of this may allah for...,1,earthquake,None,,"[deed, reason, may, allah, forgiv, us]"
1,4,NaN,NaN,forest fire near la ronge sask canada,1,None,None,,"[forest, fire, near, la, rong, sask, canada]"
2,5,NaN,NaN,all residents asked to shelter in place are ...,1,None,None,,"[resid, ask, shelter, place, notifi, offic, ev..."
3,6,NaN,NaN,13 000 people receive evacuation orders in cal...,1,wildfires,None,,"[13, 000, peopl, receiv, evacu, order, califor..."
4,7,NaN,NaN,just got sent this photo from ruby as smoke fr...,1,"alaska, wildfires",None,,"[got, sent, photo, rubi, smoke, pour, school]"


In [7]:
data_train[data_train['hashtags'].str.strip() != ""].head()

,id,keyword,location,text,target,hashtags,mentions,urls,tokens
0,1,NaN,NaN,our deeds are the reason of this may allah for...,1,earthquake,None,,"[deed, reason, may, allah, forgiv, us]"
1,4,NaN,NaN,forest fire near la ronge sask canada,1,None,None,,"[forest, fire, near, la, rong, sask, canada]"
2,5,NaN,NaN,all residents asked to shelter in place are ...,1,None,None,,"[resid, ask, shelter, place, notifi, offic, ev..."
3,6,NaN,NaN,13 000 people receive evacuation orders in cal...,1,wildfires,None,,"[13, 000, peopl, receiv, evacu, order, califor..."
4,7,NaN,NaN,just got sent this photo from ruby as smoke fr...,1,"alaska, wildfires",None,,"[got, sent, photo, rubi, smoke, pour, school]"


In [8]:
data_train[data_train['urls'].str.strip() != ""].head()

,id,keyword,location,text,target,hashtags,mentions,urls,tokens
31,48,ablaze,birmingham,wholesale markets ablaze,1,None,bbcmtd,http://t.co/lhyxeohy6c,"[wholesal, market, ablaz]"
32,49,ablaze,est. september 2012 - bristol,we always try to bring the heavy,0,"metal, rt",None,http://t.co/yao1e0xngw,"[alway, tri, bring, heavi]"
33,50,ablaze,africa,breaking news nigeria flag set ablaze in aba,1,africanbaze,None,http://t.co/2nndbgwyei,"[break, news, nigeria, flag, set, ablaz, aba]"
35,53,ablaze,"london, uk",on plus side look at the sky last night it was...,0,None,None,http://t.co/qqsmshaj3n,"[plus, side, look, sky, last, night, ablaz]"
37,55,ablaze,world wide!!,inec office in abia set ablaze,1,None,None,http://t.co/3imaomknna,"[inec, offic, abia, set, ablaz]"
